In [68]:
import pandas as pd
import numpy as np
import numpy.linalg as npl

In [50]:
df_kdd_2018_02_03 = pd.read_csv('../input/beijing_201802_201803_me.csv').drop_duplicates()
df_kdd_2018_02_03['utc_time'] = pd.to_datetime(df_kdd_2018_02_03['utc_time'])
df_kdd_2018_02_03.head()

,station_id,utc_time,weather,temperature,pressure,humidity,wind_speed,wind_direction
0,beijing_meo,2018-01-31 16:00:00,Sunny/clear,-4.8,1023.9,39,0.9,59
1,beijing_meo,2018-01-31 17:00:00,Sunny/clear,-6.1,1023.4,42,0.0,999017
2,beijing_meo,2018-01-31 18:00:00,Sunny/clear,-7.0,1023.1,45,0.5,178
3,beijing_meo,2018-01-31 19:00:00,Haze,-7.7,1022.8,47,0.0,999017
4,beijing_meo,2018-01-31 20:00:00,Sunny/clear,-6.3,1022.4,44,2.0,71


In [51]:
df_kdd_2018_02_03.utc_time.max()

Timestamp('2018-04-01 00:00:00')

In [21]:
df_aic = pd.read_csv('../input/train.csv').replace(-9999., np.nan)
df_aic['dates'] = pd.to_datetime(df_aic.dates, format='%Y%m%d%H') + df_aic.foretimes.apply(lambda x: pd.Timedelta(x, unit='h'))
df_aic.head()

,station_date_time,stations,dates,foretimes,psfc_M,t2m_M,q2m_M,rh2m_M,w10m_M,d10m_M,...,Q500_M,psur_obs,t2m_obs,q2m_obs,w10m_obs,d10m_obs,rh2m_obs,u10m_obs,v10m_obs,RAIN_obs
0,90001_2015030103_0,90001,2015-03-01 03:00:00,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1018.5,4.0,2.143617,0.8,89.0,43.0,-0.799878,-0.013962,0.0
1,90001_2015030103_1,90001,2015-03-01 04:00:00,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1017.6,5.8,2.264909,0.4,130.0,40.0,-0.306418,0.257115,0.0
2,90001_2015030103_2,90001,2015-03-01 05:00:00,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1015.8,8.0,2.043311,1.5,251.0,31.0,1.418278,0.488352,0.0
3,90001_2015030103_3,90001,2015-03-01 06:00:00,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1014.4,10.0,1.594945,3.7,264.0,21.0,3.679731,0.386757,0.0
4,90001_2015030103_4,90001,2015-03-01 07:00:00,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1013.4,10.7,1.511350,2.9,237.0,19.0,2.432145,1.579453,0.0


In [22]:
df_aic.shape

(439560, 42)

In [52]:
df_aic_corresp = df_aic[lambda x: (x.dates >= df_kdd_2018_02_03.utc_time.min()) & (x.dates <= df_kdd_2018_02_03.utc_time.max())]
df_aic_corresp.head()

,station_date_time,stations,dates,foretimes,psfc_M,t2m_M,q2m_M,rh2m_M,w10m_M,d10m_M,...,Q500_M,psur_obs,t2m_obs,q2m_obs,w10m_obs,d10m_obs,rh2m_obs,u10m_obs,v10m_obs,RAIN_obs
39492,90001_2018013103_13,90001,2018-01-31 16:00:00,13,1023.333,-4.310,1.068526,39.520,0.345,306.917,...,0.250,1024.2,-9.5,0.686761,0.8,54.0,38.0,-0.647214,-0.470228,0.0
39493,90001_2018013103_14,90001,2018-01-31 17:00:00,14,1023.160,-4.674,1.072169,40.698,0.828,307.910,...,0.240,1023.8,-10.6,0.660976,NaN,NaN,40.0,NaN,NaN,0.0
39494,90001_2018013103_15,90001,2018-01-31 18:00:00,15,1022.679,-5.318,1.058876,42.233,0.876,233.661,...,0.250,1023.5,-10.9,0.693881,0.4,39.0,43.0,-0.251728,-0.310858,0.0
39495,90001_2018013103_16,90001,2018-01-31 19:00:00,16,1022.719,-5.677,1.091749,44.736,1.080,268.343,...,0.253,1023.0,-11.7,0.711092,0.9,74.0,47.0,-0.865135,-0.248074,0.0
39496,90001_2018013103_17,90001,2018-01-31 20:00:00,17,1022.623,-5.964,1.117818,46.888,0.950,254.678,...,0.250,1022.5,-11.8,0.704647,NaN,NaN,47.0,NaN,NaN,0.0


In [53]:
df_aic_corresp.shape

(21920, 42)

In [54]:
df_aic_corresp.groupby('stations').size()

stations
90001    2192
90002    2192
90003    2192
90004    2192
90005    2192
90006    2192
90007    2192
90008    2192
90009    2192
90010    2192
dtype: int64

In [55]:
df_aic_corresp_wysf = df_aic_corresp[[
    'stations', 
    'dates', 
    't2m_obs', 
    'psur_obs', 
    'rh2m_obs', 
    'w10m_obs', 
]].drop_duplicates()  # 因为原先有睿图预测的重叠值，而两天的观测值其实是一样的
df_aic_corresp_wysf.head()

,stations,dates,t2m_obs,psur_obs,rh2m_obs,w10m_obs
39492,90001,2018-01-31 16:00:00,-9.5,1024.2,38.0,0.8
39493,90001,2018-01-31 17:00:00,-10.6,1023.8,40.0,NaN
39494,90001,2018-01-31 18:00:00,-10.9,1023.5,43.0,0.4
39495,90001,2018-01-31 19:00:00,-11.7,1023.0,47.0,0.9
39496,90001,2018-01-31 20:00:00,-11.8,1022.5,47.0,NaN


In [56]:
df_aic_corresp_wysf.groupby('stations').size()  # 每个站点都有相同的时间点

stations
90001    1425
90002    1425
90003    1425
90004    1425
90005    1425
90006    1425
90007    1425
90008    1425
90009    1425
90010    1425
dtype: int64

因为有12小时重叠，所以去除后大致变为原先的2/3，这是合理的，但最后一天和第一天也许没什么重叠，所以不是严格的2/3

In [80]:
df_kdd_2018_02_03.groupby('station_id').size()  # 除了第一个之外，其他每个站点都少一个时间点，为啥

station_id
beijing_meo        1425
chaoyang_meo       1424
daxing_meo         1424
fangshan_meo       1424
fengtai_meo        1424
hadian_meo         1424
huairou_meo        1424
mentougou_meo      1424
miyun_meo          1424
pingchang_meo      1424
pinggu_meo         1424
shangdianzi_meo    1424
shijingshan_meo    1424
shunyi_meo         1424
tongzhou_meo       1424
xiayunling_meo     1424
yanqing_meo        1424
zhaitang_meo       1424
dtype: int64

In [81]:
d_results = {}
for aic_station_id in df_aic_corresp_wysf.stations.unique():
    print('-'*10)
    print(aic_station_id)
    print('-'*10)
    df_test = df_aic_corresp_wysf[lambda x: x.stations == aic_station_id].sort_values('dates')
    for station_id in df_kdd_2018_02_03.station_id.unique():
        print(station_id)
        df_train = df_kdd_2018_02_03[lambda x: (x.station_id == station_id)].sort_values('utc_time')
        # 计算距离，求差、把nan替换为0后计算norm，因为很多站点都少一个时间点，所以这里要用isin来判断
        dist = npl.norm(np.nan_to_num(df_train[[
            'temperature', 'pressure', 'humidity', 'wind_speed']].values - \
                                      df_test.loc[lambda x: x.dates.isin(df_train.utc_time), [
            't2m_obs', 'psur_obs', 'rh2m_obs', 'w10m_obs']].values))
        print(dist)
        if dist < 1:
            print('!!!!!!')
            d_results[aic_station_id] = station_id

----------
90001
----------
beijing_meo
354.28495593236806
chaoyang_meo
394.6889534810925
daxing_meo
363.36231229999623
fangshan_meo
436.2349023175473
fengtai_meo
446.26199703761466
hadian_meo
439.24945076801185
huairou_meo
426.54057251333086
mentougou_meo
507.5513471561277
miyun_meo
453.54907121501196
pingchang_meo
556.648282131545
pinggu_meo
0.0
!!!!!!
shangdianzi_meo
1316.6174463373939
shijingshan_meo
484.6806783852643
shunyi_meo
374.8920111178685
tongzhou_meo
494.2045831434589
xiayunling_meo
1820.6066214314394
yanqing_meo
2175.907111988009
zhaitang_meo
1973.979835256683
----------
90002
----------
beijing_meo
303.21655627620333
chaoyang_meo
249.88755471211445
daxing_meo
349.25158553684474
fangshan_meo
377.09778572672633
fengtai_meo
359.01960949229505
hadian_meo
351.21920505576
huairou_meo
389.78483808378184
mentougou_meo
455.76988711410064
miyun_meo
382.12792360674194
pingchang_meo
472.8170576449205
pinggu_meo
374.9034142282516
shangdianzi_meo
1307.5538879908543
shijingshan_meo
406

In [82]:
d_results

{90001: 'pinggu_meo',
 90002: 'shunyi_meo',
 90003: 'shangdianzi_meo',
 90005: 'pingchang_meo',
 90010: 'zhaitang_meo'}

In [78]:
df_aic_corresp_wysf[lambda x: x.stations == 90004].sort_values('dates')

,stations,dates,t2m_obs,psur_obs,rh2m_obs,w10m_obs
171360,90004,2018-01-31 16:00:00,-9.5,1019.0,46.0,1.0
171361,90004,2018-01-31 17:00:00,-10.4,1018.9,52.0,1.3
171362,90004,2018-01-31 18:00:00,-10.9,1018.7,55.0,0.8
171363,90004,2018-01-31 19:00:00,-11.3,1018.4,54.0,1.5
171364,90004,2018-01-31 20:00:00,-10.2,1017.8,47.0,1.9
171365,90004,2018-01-31 21:00:00,-10.1,1017.7,45.0,1.9
171366,90004,2018-01-31 22:00:00,-10.8,1017.9,46.0,1.3
171367,90004,2018-01-31 23:00:00,-12.5,1018.7,52.0,1.4
171368,90004,2018-02-01 00:00:00,-10.7,1019.3,47.0,2.6
171369,90004,2018-02-01 01:00:00,-8.2,1019.7,39.0,2.8


In [83]:
df_kdd_2018_02_03[lambda x: (x.station_id == 'miyun_meo')].sort_values('utc_time')

,station_id,utc_time,weather,temperature,pressure,humidity,wind_speed,wind_direction
11857,miyun_meo,2018-01-31 16:00:00,Sunny/clear,-9.5,1019.0,46,1.0,88
11858,miyun_meo,2018-01-31 17:00:00,Sunny/clear,-10.4,1018.9,52,1.3,90
11859,miyun_meo,2018-01-31 18:00:00,Sunny/clear,-10.9,1018.7,55,0.8,102
11860,miyun_meo,2018-01-31 19:00:00,Sunny/clear,-11.3,1018.4,54,1.5,123
11861,miyun_meo,2018-01-31 20:00:00,Sunny/clear,-10.2,1017.8,47,1.9,89
11862,miyun_meo,2018-01-31 21:00:00,Sunny/clear,-10.1,1017.7,45,1.9,100
11863,miyun_meo,2018-01-31 22:00:00,Sunny/clear,-10.8,1017.9,46,1.3,65
11864,miyun_meo,2018-01-31 23:00:00,Sunny/clear,-12.5,1018.7,52,1.4,85
11865,miyun_meo,2018-02-01 00:00:00,Sunny/clear,-10.7,1019.3,47,2.6,87
11866,miyun_meo,2018-02-01 01:00:00,Sunny/clear,-8.2,1019.7,39,2.8,87


In [96]:
temp = df_kdd_2018_02_03[lambda x: (x.station_id == 'miyun_meo')].sort_values('utc_time')
df_test = df_aic_corresp_wysf[lambda x: (x.stations == 90004) & (x.dates.isin(temp.utc_time))].sort_values('dates')
ind = np.any((temp[['temperature', 'pressure', 'humidity', 'wind_speed']].values - \
              df_test[['t2m_obs', 'psur_obs', 'rh2m_obs', 'w10m_obs']].values) != 0, axis=1)

In [97]:
temp[ind]

,station_id,utc_time,weather,temperature,pressure,humidity,wind_speed,wind_direction
11942,miyun_meo,2018-02-04 02:00:00,Sunny/clear,-8.4,1025.3,33,2.0,86
12002,miyun_meo,2018-02-06 12:00:00,Hail,-6.0,1015.4,34,0.0,999017
12192,miyun_meo,2018-02-14 03:00:00,Sunny/clear,2.2,1014.9,18,5.7,51
12332,miyun_meo,2018-02-19 18:00:00,Hail,-3.7,1018.6,71,0.0,999017
12532,miyun_meo,2018-02-27 18:00:00,Hail,0.5,1009.0,59,0.0,999017
12533,miyun_meo,2018-02-27 19:00:00,Hail,0.2,1007.8,60,0.0,999017
12603,miyun_meo,2018-03-02 14:00:00,Hail,-1.1,997.4,50,0.0,999017
12680,miyun_meo,2018-03-05 16:00:00,Sunny/clear,-4.0,1022.6,57,0.0,999017
12774,miyun_meo,2018-03-09 10:00:00,Hail,5.6,1008.5,37,0.0,999017
12832,miyun_meo,2018-03-11 18:00:00,Hail,3.3,1008.8,48,0.0,999017


In [100]:
df_test[ind]

,stations,dates,t2m_obs,psur_obs,rh2m_obs,w10m_obs
171481,90004,2018-02-04 02:00:00,NaN,NaN,NaN,NaN
171565,90004,2018-02-06 12:00:00,-6.0,1015.4,34.0,NaN
171852,90004,2018-02-14 03:00:00,NaN,NaN,NaN,NaN
172065,90004,2018-02-19 18:00:00,-3.7,1018.6,71.0,NaN
172361,90004,2018-02-27 18:00:00,0.5,1009.0,59.0,NaN
172362,90004,2018-02-27 19:00:00,0.2,1007.8,60.0,NaN
172455,90004,2018-03-02 14:00:00,-1.1,997.4,50.0,NaN
172581,90004,2018-03-05 16:00:00,-4.0,1022.6,57.0,NaN
172710,90004,2018-03-09 10:00:00,5.6,1008.5,37.0,NaN
172805,90004,2018-03-11 18:00:00,3.3,1008.8,48.0,NaN


In [101]:
(temp[['temperature', 'pressure', 'humidity', 'wind_speed']].values - \
              df_test[['t2m_obs', 'psur_obs', 'rh2m_obs', 'w10m_obs']].values)[ind]

array([[ nan,  nan,  nan,  nan],
       [  0.,   0.,   0.,  nan],
       [ nan,  nan,  nan,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [ nan,  nan,  nan,  nan],
       [  0.,   0., -56.,   0.],
       [ nan,  nan,  nan,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [ nan,  nan,  nan,  nan],
       [  0.,   0.,   0.,  nan]])

In [102]:
df_aic_corresp_wysf[lambda x: x.stations == 90006].sort_values('dates')

,stations,dates,t2m_obs,psur_obs,rh2m_obs,w10m_obs
259272,90006,2018-01-31 16:00:00,-8.0,977.3,42.0,0.9
259273,90006,2018-01-31 17:00:00,-8.6,977.0,43.0,0.7
259274,90006,2018-01-31 18:00:00,-9.1,976.7,46.0,1.1
259275,90006,2018-01-31 19:00:00,-9.5,976.3,46.0,0.7
259276,90006,2018-01-31 20:00:00,-9.6,975.7,48.0,1.4
259277,90006,2018-01-31 21:00:00,-9.8,975.9,47.0,1.1
259278,90006,2018-01-31 22:00:00,-9.9,976.1,48.0,0.9
259279,90006,2018-01-31 23:00:00,-9.6,976.4,47.0,1.0
259280,90006,2018-02-01 00:00:00,-10.3,977.0,49.0,1.0
259281,90006,2018-02-01 01:00:00,-8.5,977.2,45.0,0.9


In [103]:
df_kdd_2018_02_03[lambda x: (x.station_id == 'xiayunling_meo')].sort_values('utc_time')

,station_id,utc_time,weather,temperature,pressure,humidity,wind_speed,wind_direction
22231,xiayunling_meo,2018-01-31 16:00:00,Sunny/clear,-8.0,977.3,42,0.9,171
22232,xiayunling_meo,2018-01-31 17:00:00,Sunny/clear,-8.6,977.0,43,0.7,163
22233,xiayunling_meo,2018-01-31 18:00:00,Sunny/clear,-9.1,976.7,46,1.1,162
22234,xiayunling_meo,2018-01-31 19:00:00,Sunny/clear,-9.5,976.3,46,0.7,167
22235,xiayunling_meo,2018-01-31 20:00:00,Sunny/clear,-9.6,975.7,48,1.4,204
22236,xiayunling_meo,2018-01-31 21:00:00,Sunny/clear,-9.8,975.9,47,1.1,106
22237,xiayunling_meo,2018-01-31 22:00:00,Sunny/clear,-9.9,976.1,48,0.9,133
22238,xiayunling_meo,2018-01-31 23:00:00,Sunny/clear,-9.6,976.4,47,1.0,151
22239,xiayunling_meo,2018-02-01 00:00:00,Sunny/clear,-10.3,977.0,49,1.0,150
22240,xiayunling_meo,2018-02-01 01:00:00,Sunny/clear,-8.5,977.2,45,0.9,123


In [104]:
df_kdd_2018_02_03[lambda x: (x.station_id == 'zhaitang_meo')].sort_values('utc_time')

,station_id,utc_time,weather,temperature,pressure,humidity,wind_speed,wind_direction
25195,zhaitang_meo,2018-01-31 16:00:00,Sunny/clear,-11.0,973.3,38,0.7,126
25196,zhaitang_meo,2018-01-31 17:00:00,Sunny/clear,-11.1,973.2,41,0.1,999017
25197,zhaitang_meo,2018-01-31 18:00:00,Sunny/clear,-12.2,973.0,43,1.7,202
25198,zhaitang_meo,2018-01-31 19:00:00,Sunny/clear,-12.6,972.8,45,0.8,87
25199,zhaitang_meo,2018-01-31 20:00:00,Sunny/clear,-13.4,972.3,48,0.4,300
25200,zhaitang_meo,2018-01-31 21:00:00,Sunny/clear,-14.0,972.4,48,0.4,245
25201,zhaitang_meo,2018-01-31 22:00:00,Sunny/clear,-14.2,972.6,50,0.9,103
25202,zhaitang_meo,2018-01-31 23:00:00,Sunny/clear,-14.5,973.0,52,0.8,221
25203,zhaitang_meo,2018-02-01 00:00:00,Sunny/clear,-14.8,973.7,50,0.7,259
25204,zhaitang_meo,2018-02-01 01:00:00,Sunny/clear,-11.7,974.0,44,0.8,237


In [108]:
temp = df_kdd_2018_02_03[lambda x: (x.station_id == 'xiayunling_meo')].sort_values('utc_time')
df_test = df_aic_corresp_wysf[lambda x: (x.stations == 90006) & (x.dates.isin(temp.utc_time))].sort_values('dates')
ind = np.any((temp[['temperature', 'pressure', 'humidity', 'wind_speed']].values - \
              df_test[['t2m_obs', 'psur_obs', 'rh2m_obs', 'w10m_obs']].values) != 0, axis=1)
(temp[['temperature', 'pressure', 'humidity', 'wind_speed']].values - \
              df_test[['t2m_obs', 'psur_obs', 'rh2m_obs', 'w10m_obs']].values)[ind]

array([[  0.,   0.,   0.,  nan],
       [ nan,  nan,  nan,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [ nan,  nan,  nan,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  0.,   0.,   0.,  nan],
       [  

In [112]:
temp[ind]

,station_id,utc_time,weather,temperature,pressure,humidity,wind_speed,wind_direction
22281,xiayunling_meo,2018-02-02 17:00:00,Sunny/clear,-12.2,986.3,39,0.0,999017
22316,xiayunling_meo,2018-02-04 02:00:00,Sunny/clear,-4.8,981.3,26,2.8,120
22365,xiayunling_meo,2018-02-06 01:00:00,Sunny/clear,-11.9,979.2,49,0.0,999017
22414,xiayunling_meo,2018-02-08 01:00:00,Sunny/clear,-10.6,971.4,39,0.0,999017
22433,xiayunling_meo,2018-02-08 20:00:00,Sunny/clear,-9.9,965.6,42,0.0,999017
22436,xiayunling_meo,2018-02-08 23:00:00,Sunny/clear,-10.1,966.8,45,0.0,999017
22539,xiayunling_meo,2018-02-13 01:00:00,Sunny/clear,-5.8,965.7,32,0.0,999017
22566,xiayunling_meo,2018-02-14 03:00:00,Sunny/clear,3.0,971.8,18,1.8,15
22574,xiayunling_meo,2018-02-14 11:00:00,Sunny/clear,-1.7,973.8,25,0.0,999017
22576,xiayunling_meo,2018-02-14 13:00:00,Sunny/clear,-4.3,975.9,31,0.0,999017


In [113]:
df_test[ind]

,stations,dates,t2m_obs,psur_obs,rh2m_obs,w10m_obs
259347,90006,2018-02-02 17:00:00,-12.2,986.3,39.0,NaN
259393,90006,2018-02-04 02:00:00,NaN,NaN,NaN,NaN
259466,90006,2018-02-06 01:00:00,-11.9,979.2,49.0,NaN
259540,90006,2018-02-08 01:00:00,-10.6,971.4,39.0,NaN
259572,90006,2018-02-08 20:00:00,-9.9,965.6,42.0,NaN
259575,90006,2018-02-08 23:00:00,-10.1,966.8,45.0,NaN
259725,90006,2018-02-13 01:00:00,-5.8,965.7,32.0,NaN
259764,90006,2018-02-14 03:00:00,NaN,NaN,NaN,NaN
259772,90006,2018-02-14 11:00:00,-1.7,973.8,25.0,NaN
259774,90006,2018-02-14 13:00:00,-4.3,975.9,31.0,NaN


感觉都是湿度那一列有问题，而且可以确定都是aic数据的问题

可能湿度恰好为100的都有问题？需要密切关注

可能另三个值都为nan时就会出问题？但逆命题并不成立，并不是所有出问题的湿度、另三个值都为nan，比如前面90004那个时间点就不是

In [122]:
d_results = {}
for aic_station_id in df_aic_corresp_wysf.stations.unique():
    print('-'*10)
    print(aic_station_id)
    print('-'*10)
    df_test = df_aic_corresp_wysf[lambda x: x.stations == aic_station_id].sort_values('dates')
    for station_id in df_kdd_2018_02_03.station_id.unique():
        print(station_id)
        df_train = df_kdd_2018_02_03[lambda x: (x.station_id == station_id)].sort_values('utc_time')
        # 计算距离，求差、把nan替换为0后计算norm，因为很多站点都少一个时间点，所以这里要用isin来判断
        dist = npl.norm(np.nan_to_num(df_train[[
            'temperature', 'pressure', 'wind_speed']].values - \
                                      df_test.loc[lambda x: x.dates.isin(df_train.utc_time), [
            't2m_obs', 'psur_obs', 'w10m_obs']].values))
        print(dist)
        if dist < 1:
            print('!!!!!!')
            d_results[aic_station_id] = (station_id, dist)

----------
90001
----------
beijing_meo
108.54413848752951
chaoyang_meo
115.123281746135
daxing_meo
108.7573905534699
fangshan_meo
136.179624026504
fengtai_meo
167.78191201676057
hadian_meo
132.51822516167348
huairou_meo
222.77984648526902
mentougou_meo
274.65500177495403
miyun_meo
207.9008417491377
pingchang_meo
238.42883634325784
pinggu_meo
0.0
!!!!!!
shangdianzi_meo
1224.7932478585926
shijingshan_meo
192.32358149743365
shunyi_meo
98.19887983067828
tongzhou_meo
98.49451761392604
xiayunling_meo
1756.8009192848235
yanqing_meo
2108.434433412621
zhaitang_meo
1900.7988820493347
----------
90002
----------
beijing_meo
72.50020689625649
chaoyang_meo
71.94991313406844
daxing_meo
92.17195885951426
fangshan_meo
129.04937039753423
fengtai_meo
151.2384871651392
hadian_meo
112.96428639176186
huairou_meo
235.13234571194155
mentougou_meo
277.2908040307143
miyun_meo
223.94809666527647
pingchang_meo
234.30742625875098
pinggu_meo
98.2424042865401
shangdianzi_meo
1241.305429779472
shijingshan_meo
184.8

In [123]:
d_results

{90001: ('pinggu_meo', 0.0),
 90002: ('shunyi_meo', 0.0),
 90003: ('shangdianzi_meo', 0.0),
 90004: ('miyun_meo', 0.0),
 90005: ('pingchang_meo', 0.0),
 90006: ('xiayunling_meo', 0.0),
 90009: ('beijing_meo', 0.0),
 90010: ('zhaitang_meo', 0.22360679774997977)}

In [116]:
df_aic_corresp_wysf[lambda x: x.stations == 90007].sort_values('dates')

,stations,dates,t2m_obs,psur_obs,rh2m_obs,w10m_obs
303228,90007,2018-01-31 16:00:00,-13.0,986.7,43.0,0.9
303229,90007,2018-01-31 17:00:00,-14.1,986.6,46.0,0.9
303230,90007,2018-01-31 18:00:00,-14.9,986.6,48.0,0.8
303231,90007,2018-01-31 19:00:00,-15.7,986.4,52.0,1.1
303232,90007,2018-01-31 20:00:00,-16.2,986.0,53.0,1.0
303233,90007,2018-01-31 21:00:00,-16.6,985.9,54.0,1.5
303234,90007,2018-01-31 22:00:00,-17.3,986.1,57.0,1.2
303235,90007,2018-01-31 23:00:00,-17.6,986.7,59.0,1.4
303236,90007,2018-02-01 00:00:00,-17.6,987.3,59.0,1.2
303237,90007,2018-02-01 01:00:00,-16.0,988.0,56.0,0.4


In [117]:
df_kdd_2018_02_03[lambda x: (x.station_id == 'shangdianzi_meo')].sort_values('utc_time')

,station_id,utc_time,weather,temperature,pressure,humidity,wind_speed,wind_direction
16303,shangdianzi_meo,2018-01-31 16:00:00,Sunny/clear,-10.5,991.0,38,0.8,192
16304,shangdianzi_meo,2018-01-31 17:00:00,Sunny/clear,-11.2,990.8,40,0.7,87
16305,shangdianzi_meo,2018-01-31 18:00:00,Sunny/clear,-11.5,990.6,41,0.9,68
16306,shangdianzi_meo,2018-01-31 19:00:00,Sunny/clear,-12.5,990.3,44,1.0,99
16307,shangdianzi_meo,2018-01-31 20:00:00,Sunny/clear,-12.8,989.9,45,2.4,46
16308,shangdianzi_meo,2018-01-31 21:00:00,Sunny/clear,-12.8,989.7,45,3.7,53
16309,shangdianzi_meo,2018-01-31 22:00:00,Sunny/clear,-13.2,989.7,46,2.8,50
16310,shangdianzi_meo,2018-01-31 23:00:00,Sunny/clear,-13.6,990.5,47,2.1,67
16311,shangdianzi_meo,2018-02-01 00:00:00,Sunny/clear,-13.2,991.2,47,2.6,60
16312,shangdianzi_meo,2018-02-01 01:00:00,Sunny/clear,-10.7,991.7,40,4.0,53


In [118]:
df_kdd_2018_02_03[lambda x: (x.station_id == 'yanqing_meo')].sort_values('utc_time')

,station_id,utc_time,weather,temperature,pressure,humidity,wind_speed,wind_direction
23713,yanqing_meo,2018-01-31 16:00:00,Sunny/clear,-9.8,967.2,35,1.5,87
23714,yanqing_meo,2018-01-31 17:00:00,Sunny/clear,-10.2,967.0,38,0.8,116
23715,yanqing_meo,2018-01-31 18:00:00,Sunny/clear,-11.0,966.9,43,1.0,93
23716,yanqing_meo,2018-01-31 19:00:00,Sunny/clear,-12.2,966.6,43,1.6,80
23717,yanqing_meo,2018-01-31 20:00:00,Sunny/clear,-12.7,966.2,44,1.1,121
23718,yanqing_meo,2018-01-31 21:00:00,Sunny/clear,-13.5,966.3,47,0.7,9
23719,yanqing_meo,2018-01-31 22:00:00,Sunny/clear,-13.3,966.6,45,1.0,103
23720,yanqing_meo,2018-01-31 23:00:00,Sunny/clear,-13.3,966.9,48,0.8,132
23721,yanqing_meo,2018-02-01 00:00:00,Sunny/clear,-13.3,967.7,48,0.5,145
23722,yanqing_meo,2018-02-01 01:00:00,Sunny/clear,-11.8,968.2,48,0.8,240


7和8是真找不到了，7还有一些较小的，可能在较远的郊区，可能在东北或西南，因为距离较小的都在这些区域，但8距离其他的都非常大，不知道为什么

In [119]:
df_aic_corresp_wysf[lambda x: x.stations == 90008].sort_values('dates')

,stations,dates,t2m_obs,psur_obs,rh2m_obs,w10m_obs
347184,90008,2018-01-31 16:00:00,-10.7,879.7,34.0,3.4
347185,90008,2018-01-31 17:00:00,-10.7,879.4,35.0,3.4
347186,90008,2018-01-31 18:00:00,-11.0,879.3,36.0,1.9
347187,90008,2018-01-31 19:00:00,-11.7,878.9,39.0,4.1
347188,90008,2018-01-31 20:00:00,-11.6,878.5,40.0,4.1
347189,90008,2018-01-31 21:00:00,-11.7,878.4,42.0,3.4
347190,90008,2018-01-31 22:00:00,-12.4,878.6,44.0,3.3
347191,90008,2018-01-31 23:00:00,-12.4,878.9,46.0,4.0
347192,90008,2018-02-01 00:00:00,-12.1,879.4,46.0,4.5
347193,90008,2018-02-01 01:00:00,-11.4,879.9,44.0,6.1


感觉好像是气压明显偏低？可以去掉气压试试

另外，有的不是完全为0的，仍然可以找到是哪些列有问题，然后看看是aic的问题，还是kdd的问题，aic有问题的概率居多，这样的话就可以想办法修正

In [124]:
d_results = {}
for aic_station_id in df_aic_corresp_wysf.stations.unique():
    print('-'*10)
    print(aic_station_id)
    print('-'*10)
    df_test = df_aic_corresp_wysf[lambda x: x.stations == aic_station_id].sort_values('dates')
    for station_id in df_kdd_2018_02_03.station_id.unique():
        print(station_id)
        df_train = df_kdd_2018_02_03[lambda x: (x.station_id == station_id)].sort_values('utc_time')
        # 计算距离，求差、把nan替换为0后计算norm，因为很多站点都少一个时间点，所以这里要用isin来判断
        dist = npl.norm(np.nan_to_num(df_train[[
            'temperature', 'wind_speed']].values - \
                                      df_test.loc[lambda x: x.dates.isin(df_train.utc_time), [
            't2m_obs', 'w10m_obs']].values))
        print(dist)
        if dist < 1:
            print('!!!!!!')
            d_results[aic_station_id] = (station_id, dist)

----------
90001
----------
beijing_meo
107.5899623570898
chaoyang_meo
113.42790661913848
daxing_meo
102.48370602198186
fangshan_meo
108.14129645977063
fengtai_meo
124.04773274832554
hadian_meo
108.12788724468818
huairou_meo
83.84002624045391
mentougou_meo
113.0324732101355
miyun_meo
95.35192709117106
pingchang_meo
115.88787684654507
pinggu_meo
0.0
!!!!!!
shangdianzi_meo
125.34727759309335
shijingshan_meo
115.80526758312853
shunyi_meo
95.90990564065842
tongzhou_meo
80.94844038028157
xiayunling_meo
105.95706677706778
yanqing_meo
124.71643837121073
zhaitang_meo
130.80756858836568
----------
90002
----------
beijing_meo
69.53941328484157
chaoyang_meo
66.32103738633768
daxing_meo
77.21839159164091
fangshan_meo
84.70407310159294
fengtai_meo
81.1887923299762
hadian_meo
68.10814929213684
huairou_meo
78.7716954241814
mentougou_meo
80.2646248356024
miyun_meo
101.22148981318148
pingchang_meo
71.95074704268191
pinggu_meo
95.95446836911766
shangdianzi_meo
136.41730095556062
shijingshan_meo
76.6093

In [125]:
d_results

{90001: ('pinggu_meo', 0.0),
 90002: ('shunyi_meo', 0.0),
 90003: ('shangdianzi_meo', 0.0),
 90004: ('miyun_meo', 0.0),
 90005: ('pingchang_meo', 0.0),
 90006: ('xiayunling_meo', 0.0),
 90009: ('beijing_meo', 0.0),
 90010: ('zhaitang_meo', 0.22360679774997977)}

还是不行，仍然差得非常大

In [126]:
temp = df_kdd_2018_02_03[lambda x: (x.station_id == 'zhaitang_meo')].sort_values('utc_time')
df_test = df_aic_corresp_wysf[lambda x: (x.stations == 90010) & (x.dates.isin(temp.utc_time))].sort_values('dates')
ind = np.any((temp[['temperature', 'pressure', 'humidity', 'wind_speed']].values - \
              df_test[['t2m_obs', 'psur_obs', 'rh2m_obs', 'w10m_obs']].values) != 0, axis=1)
(temp[['temperature', 'pressure', 'humidity', 'wind_speed']].values - \
              df_test[['t2m_obs', 'psur_obs', 'rh2m_obs', 'w10m_obs']].values)[ind]

array([[ 0. ,  0. ,  0. ,  nan],
       [ nan,  nan,  nan,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ nan,  nan,  nan,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0. ,  0. ,  0. ,  nan],
       [ 0

只是其中一条的温度和风速有一点点偏差，问题不大

In [128]:
df_aic_corresp_wysf.groupby('stations').agg([np.mean, np.std])

t2m_obs               psur_obs             rh2m_obs             \
              mean       std         mean       std       mean        std   
stations                                                                    
90001     2.627586  8.229853  1018.372836  8.038774  38.952146  19.878021   
90002     3.668473  7.656376  1018.783392  8.064906  37.913441  18.883713   
90003     1.385714  8.725321   986.058550  7.317128  36.383533  17.910064   
90004     2.134155  8.487532  1013.495282  7.952284  39.637324  19.688022   
90005     3.997115  7.534107  1012.875792  7.927740  32.104856  16.890730   
90006     2.240890  7.960060   971.874929  6.953513  37.779422  18.676595   
90007    -0.263969  9.845747   981.396622  7.332411  36.584799  18.016679   
90008    -2.614638  8.995707   877.507460  5.097993  37.959184  18.444275   
90009     4.108121  7.500263  1018.319209  8.084035  37.837914  19.791129   
90010     1.525053  8.833644   968.078732  6.935510  34.783955  18.162465   

          w10m_obs            
              mean       std  
stations                      
90001     2.071691  1.472140  
90002     1.944267  1.315391  
90003     2.879335  1.769204  
90004     1.618492  1.062683  
90005     2.069160  1.387867  
90006     1.849927  1.337574  
90007     2.679096  2.014039  
90008     4.238001  2.010439  
90009     2.229488  1.410511  
90010     2.113318  1.624297

可以看到，7和8的气温明显更低，而且8的气压也明显更低、风速明显更高，所以它们也必须各自建立模型才好，只不过就没法用附近站点的信息作为特征了，不过它们的差别这么大，也许也不适合用附近站点信息作为特征

KDD那个只有18个站点，对比北京20个站点可以看到，少了佛爷顶和汤河口，对照平均气温来看，8应该是佛爷顶，7可能是汤河口，这个可以找相关数据看看

还是要看看官方baseline，有没有对每个站点分别建模，有没有可以借鉴的思路和方法